In [22]:

import zstandard
import os
import json
import csv
import sys
import time
from datetime import datetime
from google.colab import drive

# Mount Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- CONFIGURATION ---

# Paths (write directly to Google Drive for reliability)
BASE_DIR = '/content/drive/MyDrive/СSS_data/raw/reddit'
SUBMISSIONS_DIR = os.path.join(BASE_DIR, 'submissions')
COMMENTS_DIR = os.path.join(BASE_DIR, 'comments')

# Output files also go to Drive so they do not disappear on disconnect
OUTPUT_DIR = '/content/drive/MyDrive/CSS_data/processed' # Create this folder or the code will create it itself
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

OUT_SUBMISSIONS = os.path.join(OUTPUT_DIR, 'filtered_submissions.csv')
OUT_COMMENTS = os.path.join(OUTPUT_DIR, 'filtered_comments.csv')

# Batch writing settings
BATCH_SIZE = 5000  # Write to disk every 5000 matched rows

FILES_SUBMISSIONS = ['RS_2025-06.zst', 'RS_2025-08.zst']
FILES_COMMENTS = ['RC_2025-06.zst']

# Target subreddit list
raw_targets = [
    "ukraine", "UkraineConflict", "poland", "AskConservatives", "geopolitics",
    "conspiracy", "europe", "worldnews",
    'ukraine', 'ukrainianconflict', 'actionforukraine',
    'russiaukrainewar2022', 'slavaukrayini', 'freedomofrussia',
    'volunteersforukraine', 'supportforukraine',
    'europe', 'europeanfederalists', 'yurop', 'askeurope',
    'poland', 'polska', 'germany', 'de', 'unitedkingdom', 'ukpolitics',
    'balticstates', 'lithuania', 'latvia', 'eesti', 'finland', 'suomi',
    'worldnews', 'anime_titties', 'geopolitics', 'combatfootage',
]
TARGET_SUBREDDITS = set(sub.lower() for sub in raw_targets)

print(f"✅ Configuration is ready.")
print(f"📁 Results will be saved here: {OUTPUT_DIR}")

✅ Configuration is ready.
📁 Results will be saved here: /content/drive/MyDrive/CSS_data/processed


In [20]:
# @title 2. Optimized functions (with buffering)

def read_lines_zst(file_name):
    """Stream ZST file line by line."""
    with open(file_name, 'rb') as file_handle:
        dctx = zstandard.ZstdDecompressor(max_window_size=2**31)
        with dctx.stream_reader(file_handle) as reader:
            import io
            text_reader = io.TextIOWrapper(reader, encoding='utf-8')
            for line in text_reader:
                yield line

def process_files_buffered(file_list, input_dir, output_csv, file_type):
    """
    Processes files with buffering and append mode.
    """

    # Define headers
    if file_type == 'submission':
        headers = ['id', 'author', 'created_utc', 'subreddit', 'title', 'selftext',
                   'score', 'upvote_ratio', 'num_comments', 'url', 'domain', 'permalink']
    else: # comment file
        headers = ['id', 'link_id', 'parent_id', 'author', 'created_utc', 'subreddit',
                   'body', 'score', 'permalink']

    # Check if file already exists to decide whether to write the header
    file_exists = os.path.exists(output_csv)
    mode = 'a' if file_exists else 'w'

    print(f"🚀 Starting to write to {output_csv} (Mode: {mode})")

    total_saved = 0
    buffer = [] # Тимчасове сховище рядків

    # Open the file once and keep it open
    with open(output_csv, mode, newline='', encoding='utf-8') as f_out:
        writer = csv.writer(f_out)

        # Write header only if the file is new
        if not file_exists:
            writer.writerow(headers)

        for zst_file in file_list:
            full_path = os.path.join(input_dir, zst_file)
            print(f"\n--- Обробка файлу: {zst_file} ---")

            if not os.path.exists(full_path):
                print(f"⚠️ File not found: {full_path}")
                continue

            lines_scanned = 0
            file_saved_count = 0
            start_time = time.time()

            try:
                for line in read_lines_zst(full_path):
                    lines_scanned += 1
                    try:
                        obj = json.loads(line)
                        if obj.get('subreddit', '').lower() in TARGET_SUBREDDITS:

                            # Build the row (same as before)
                            if file_type == 'submission':
                                row = [
                                    obj.get('id'), obj.get('author'), obj.get('created_utc'),
                                    obj.get('subreddit'), obj.get('title'), obj.get('selftext'),
                                    obj.get('score'), obj.get('upvote_ratio'), obj.get('num_comments'),
                                    obj.get('url'), obj.get('domain'), obj.get('permalink')
                                ]
                            else:
                                row = [
                                    obj.get('id'), obj.get('link_id'), obj.get('parent_id'),
                                    obj.get('author'), obj.get('created_utc'), obj.get('subreddit'),
                                    obj.get('body'), obj.get('score'), obj.get('permalink')
                                ]

                            buffer.append(row)
                            file_saved_count += 1
                            total_saved += 1

                            # --- SAVE MAGIC ---
                            # When the buffer fills up, flush it to disk
                            if len(buffer) >= BATCH_SIZE:
                                writer.writerows(buffer)
                                f_out.flush() # Force flush to disk
                                os.fsync(f_out.fileno()) # OS-level guarantee
                                buffer = [] # Clear the buffer
                                print(f"💾 [Checkpoint saved] Total: {total_saved:,} rows (Time: {time.time()-start_time:.1f}s)")
                                start_time = time.time() # Reset timer for logging

                    except (json.JSONDecodeError, KeyError):
                        continue

                    if lines_scanned % 500000 == 0:
                        print(f"👀 Scanned: {lines_scanned:,} | Found: {file_saved_count:,}")

                # Flush remaining buffer after finishing the file
                if buffer:
                    writer.writerows(buffer)
                    f_out.flush()
                    buffer = []

            except Exception as e:
                print(f"❌ Error in file {zst_file}: {e}")
                # Even on error, try to save what is in the buffer
                if buffer:
                    writer.writerows(buffer)
                    buffer = []

    print(f"\n✅ COMPLETED {file_type}. Total rows in CSV: {total_saved:,}")

In [23]:
# @title 3. Run (separately for control)

# Recommend running sequentially; if the runtime crashes you will know where it stopped.

# 1. Run posts
process_files_buffered(FILES_SUBMISSIONS, SUBMISSIONS_DIR, OUT_SUBMISSIONS, 'submission')

🚀 Починаємо запис у /content/drive/MyDrive/CSS_data/processed/filtered_submissions.csv (Режим: a)

--- Обробка файлу: RS_2025-06.zst ---
👀 Проскановано: 500,000 | Знайдено: 588
👀 Проскановано: 1,000,000 | Знайдено: 1,630
👀 Проскановано: 1,500,000 | Знайдено: 2,401
👀 Проскановано: 2,000,000 | Знайдено: 3,679
👀 Проскановано: 2,500,000 | Знайдено: 4,548
💾 [Збережено чекпоінт] Всього: 5,000 рядків (Час: 285.9s)
👀 Проскановано: 3,000,000 | Знайдено: 5,008
👀 Проскановано: 3,500,000 | Знайдено: 6,144
👀 Проскановано: 4,000,000 | Знайдено: 6,990
👀 Проскановано: 4,500,000 | Знайдено: 7,696
👀 Проскановано: 5,000,000 | Знайдено: 8,757
👀 Проскановано: 5,500,000 | Знайдено: 9,447
💾 [Збережено чекпоінт] Всього: 10,000 рядків (Час: 277.8s)
👀 Проскановано: 6,000,000 | Знайдено: 10,365
👀 Проскановано: 6,500,000 | Знайдено: 11,385
👀 Проскановано: 7,000,000 | Знайдено: 12,003
👀 Проскановано: 7,500,000 | Знайдено: 13,028
👀 Проскановано: 8,000,000 | Знайдено: 13,765
👀 Проскановано: 8,500,000 | Знайдено: 14,

In [24]:
# 2. Run comments (this will take the longest)
process_files_buffered(FILES_COMMENTS, COMMENTS_DIR, OUT_COMMENTS, 'comment')

🚀 Починаємо запис у /content/drive/MyDrive/CSS_data/processed/filtered_comments.csv (Режим: a)

--- Обробка файлу: RC_2025-06.zst ---
👀 Проскановано: 500,000 | Знайдено: 1,636
👀 Проскановано: 1,000,000 | Знайдено: 3,136
👀 Проскановано: 1,500,000 | Знайдено: 4,386
💾 [Збережено чекпоінт] Всього: 5,000 рядків (Час: 67.3s)
👀 Проскановано: 2,000,000 | Знайдено: 5,866
👀 Проскановано: 2,500,000 | Знайдено: 7,799
💾 [Збережено чекпоінт] Всього: 10,000 рядків (Час: 40.7s)
👀 Проскановано: 3,000,000 | Знайдено: 12,431
💾 [Збережено чекпоінт] Всього: 15,000 рядків (Час: 14.8s)
👀 Проскановано: 3,500,000 | Знайдено: 18,596
💾 [Збережено чекпоінт] Всього: 20,000 рядків (Час: 28.9s)
💾 [Збережено чекпоінт] Всього: 25,000 рядків (Час: 16.4s)
👀 Проскановано: 4,000,000 | Знайдено: 25,135
💾 [Збережено чекпоінт] Всього: 30,000 рядків (Час: 15.1s)
👀 Проскановано: 4,500,000 | Знайдено: 31,376
💾 [Збережено чекпоінт] Всього: 35,000 рядків (Час: 15.7s)
👀 Проскановано: 5,000,000 | Знайдено: 37,072
💾 [Збережено чекпо